In [206]:
#[(phase, Vector of string with the Gate name and qubit indices)]
#implement a function that takes in a list of Pauli strings and builds a ZX graph with W inputs and outputs using method from https://arxiv.org/pdf/2212.04462

In [207]:
import pyzx as zx
import sympy as sp
import numpy as np

In [208]:
# pip install openfermion openfermionpyscf pyscf
from openfermionpyscf import run_pyscf
from openfermion.transforms import jordan_wigner
from openfermion.utils import get_fermion_operator

geometry = [('H', (0.0, 0.0, 0.0)),
            ('H', (0.0, 0.0, 0.7414))]  # ~equilibrium bond length in Å
basis = 'sto-3g'
multiplicity = 1
charge = 0

molecule = run_pyscf(geometry, basis, multiplicity, charge, run_scf=True, run_mp2=False, run_ccsd=False)
fermion_ham = get_fermion_operator(molecule.get_molecular_hamiltonian())
qubit_ham = jordan_wigner(fermion_ham)

print("Electronic energy (HF):", molecule.hf_energy)
print("Fermion Hamiltonian:\n", fermion_ham)
print("JW Qubit Hamiltonian:\n", qubit_ham)


ModuleNotFoundError: No module named 'openfermionpyscf'

In [210]:
pauli_string = [(1.0,["X0","X1"]),(1.0,["X1","X2"]),(-1.0,["Z0"]),(-1.0,["Z1"]),(-1.0,["Z2"])]

In [211]:
def w_spider(outputs=1): return zx.generate.spider('W', 1, outputs)

In [212]:
total_qubits = 0

for term in pauli_string:
    # Get the number from the gate names to determine the number of qubits
    qubit_indices = [int(gate[1:]) for gate in term[1]]
    max_index = max(qubit_indices)
    if max_index > total_qubits:
        total_qubits = max_index
    print("Alpha:", term[0], "Gates:", term[1])
    
total_qubits += 1  # Adjust for 0-based indexing
print("Total qubits needed:", total_qubits)

Alpha: 1.0 Gates: ['X0', 'X1']
Alpha: 1.0 Gates: ['X1', 'X2']
Alpha: -1.0 Gates: ['Z0']
Alpha: -1.0 Gates: ['Z1']
Alpha: -1.0 Gates: ['Z2']
Total qubits needed: 3


In [213]:
main_graph = zx.Graph()
inps = []
for q in range(total_qubits):
    in_vertex = main_graph.add_vertex(zx.VertexType.BOUNDARY, qubit=q, row=0)
    inps.append(in_vertex)
main_graph.set_inputs(inps)
# zx.draw(main_graph, labels=True)

In [214]:
current_row = 1  # Start adding operations from row 1
z_vertices_to_connect = []
for term in pauli_string:
    phase = term[0]
    gates = term[1]
    curr_list = []
    for gate in gates:
        gate_type = gate[0]
        qubit_index = int(gate[1:])
        
        row_increment = 0
        
        if gate_type == 'X':
            first_hadamard = main_graph.add_vertex(zx.VertexType.H_BOX, qubit=qubit_index, row=current_row)
            z_vertex = main_graph.add_vertex(zx.VertexType.Z, qubit=qubit_index, row=current_row + 1)
            second_hadamard = main_graph.add_vertex(zx.VertexType.H_BOX, qubit=qubit_index, row=current_row + 2)

            main_graph.add_edge((first_hadamard, z_vertex))
            main_graph.add_edge((z_vertex, second_hadamard))
            
            row_increment += 3
            
            curr_list.append(z_vertex)
        elif gate_type == 'Z':
            z_vertex = main_graph.add_vertex(zx.VertexType.Z, qubit=qubit_index, row=current_row)
            row_increment += 1
            
            curr_list.append(z_vertex)
        elif gate_type == 'Y':
            x_vertex_one = main_graph.add_vertex(zx.VertexType.X, qubit=qubit_index, row=current_row, phase=sp.pi/2)
            z_vertex = main_graph.add_vertex(zx.VertexType.Z, qubit=qubit_index, row=current_row + 1)
            x_vertex_two = main_graph.add_vertex(zx.VertexType.X, qubit=qubit_index, row=current_row + 2, phase=-sp.pi/2)
            
            main_graph.add_edge((x_vertex_one, z_vertex))
            main_graph.add_edge((z_vertex, x_vertex_two))
            
            row_increment += 3
            
            curr_list.append(z_vertex)
            
        print("Adding gate:", gate_type, "on qubit", qubit_index)
        
    z_vertices_to_connect.append(curr_list)
        
    current_row += row_increment
print(z_vertices_to_connect)

Adding gate: X on qubit 0
Adding gate: X on qubit 1
Adding gate: X on qubit 1
Adding gate: X on qubit 2
Adding gate: Z on qubit 0
Adding gate: Z on qubit 1
Adding gate: Z on qubit 2
[[4, 7], [10, 13], [15], [16], [17]]


In [215]:
outs = []
for q in range(total_qubits):
    out_vertex = main_graph.add_vertex(zx.VertexType.BOUNDARY, qubit=q, row=current_row)
    outs.append(out_vertex)
main_graph.set_outputs(outs)

In [216]:
for q in range(total_qubits):
    #find all vertices for qubit q
    vertices_on_qubit = [v for v in main_graph.vertices() if main_graph.qubit(v) == q]
    edges = main_graph.edge_set()
    
    # run through vertices and check wich are not connected
    for i in range(len(vertices_on_qubit)):
        v1 = vertices_on_qubit[i]
        
        if i+1 >= len(vertices_on_qubit):
            break
        
        v2 = vertices_on_qubit[i+1]
        if (v1, v2) not in edges and (v2, v1) not in edges:
            main_graph.add_edge((v1, v2))

row_range = range(len(z_vertices_to_connect))
argsorted = np.argsort([z[0] for z in z_vertices_to_connect])
print(argsorted)
row_range = [row_range[i] for i in argsorted]
print(row_range)
pauli_string_x_vertex = []
for i, z in enumerate(z_vertices_to_connect):
    corresponding_x = main_graph.add_vertex(zx.VertexType.X, qubit=-1, row=row_range[i]+6)
    for v in z:
        main_graph.add_edge((v, corresponding_x))
    pauli_string_x_vertex.append(corresponding_x)

h = list(main_graph.vertices())[-1]

root_x_vertex = main_graph.add_vertex(zx.VertexType.X, qubit=-8, row=main_graph.row(h)+1, phase=1)

top_zs = []
for x_v in pauli_string_x_vertex:
    top_zs.append(main_graph.add_vertex(zx.VertexType.Z, qubit=main_graph.qubit(x_v)-1, row=main_graph.row(x_v)))
    

[0 1 2 3 4]
[0, 1, 2, 3, 4]


In [217]:
zx.draw(main_graph, labels=True)

file_name = "output.txt"


with open(file_name, 'w') as f:
    print(main_graph.to_tikz('w_spider_circuit.tex'), file=f)

In [218]:
top_graph = zx.Graph()

w_input = top_graph.add_vertex(zx.VertexType.W_INPUT, qubit=0, row=1)
w_output = top_graph.add_vertex(zx.VertexType.W_OUTPUT, qubit=0, row=2)
top_graph.add_edge((root_x_vertex, w_input))
top_graph.add_edge((w_input, w_output)) 

w_labels = {w_input: "W_IN", w_output: "W_OUT"}

x_vertex_to_connect = []

for i, elm in enumerate(pauli_string):
    h = top_graph.add_vertex(zx.VertexType.Z_BOX, qubit=3, row=i+1, phase=elm[0] + 0j)
    x = top_graph.add_vertex(zx.VertexType.X, qubit=4, row=i+1)
    
    top_graph.add_edge((w_output, h))
    top_graph.add_edge((h, x), edgetype=zx.EdgeType.HADAMARD)
    
    x_vertex_to_connect.append(x)

KeyError: 26

In [ ]:
zx.draw(top_graph, labels=w_labels)

In [ ]:
top_graph_verts = len(top_graph.vertices())
print("Top graph vertices:", top_graph_verts)
# Shift z_vertices_to_connect indices to account for vertices in top_graph when tensoring
z_vertices_to_connect = [[v + top_graph_verts for v in lst] for lst in z_vertices_to_connect]
print("Shifted z_vertices_to_connect:", z_vertices_to_connect)
print("X vertices to connect:", x_vertex_to_connect)

tot_graph = top_graph.tensor(main_graph)

for i in range(len(x_vertex_to_connect)):
    for j in range(len(z_vertices_to_connect[i])):
        tot_graph.add_edge((x_vertex_to_connect[i], z_vertices_to_connect[i][j]))
zx.draw(tot_graph, labels=True)

Top graph vertices: 13
Shifted z_vertices_to_connect: [[17, 20], [23, 26], [28], [29], [30]]
X vertices to connect: [4, 6, 8, 10, 12]
